# Исследование данных сервиса “Яндекс.Музыка” — сравнение пользователей двух городов

**Описание проекта**. На реальных данных Яндекс.Музыки c помощью библиотеки Pandas и её возможностей проводится проверка данных и сравнение поведения и предпочтения пользователей двух столиц — Москвы и Санкт-Петербурга.

**Ход исследования:**

1. Обзор данных;
2. Предобработка данных;
3. Проверка гипотез;
4. Итоги исследования.

## 1. Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('C:/Users/alkon/practicum_projects/music_project/yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов с типом данных во всех столбцах — `object`. Видим, что в столбцах различается количество значений: в данных есть пропуски. Названия столбцов необходимо привести к единому регистру, удалить лишние пробелы и преобразовать в *snake case*.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

**Вывод.** Каждый столбец таблицы содержит данные о прослушанном треке, включая описание самой композиции (название, исполнитель, жанр) и описание пользователя (идентификатор, город, время прослушивания). На этапе предобработки необходимо обработать пропуски и привести названия столбцов к единому стилю.

## 2. Предобработка данных

Приведем названия столбцов к единому стилю.

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование столбцов
df = df.rename(
    columns={
        '  userID':'user_id',
        'Track':'track',
        '  City  ':'city',
        'Day':'day',
    }
)

# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Обработаем пропуски.

In [7]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в столбцах `track`, `genre` и `artist` заменим на идентификатор *'unknown'*. Поскольку в исследовании используется столбец `genre`, а установить причину пропусков и восстановить данные не представляется возможным, далее оценим, насколько пропуски в этом столбце повредят расчетам.

In [8]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track','artist','genre']
for c in columns_to_replace:
    df[c] = df[c].fillna('unknown')
    
# подсчёт пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Проверим данные на дубликаты.

In [9]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [10]:
# удаление явных дубликатов
df = df.drop_duplicates()
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

In [11]:
# просмотр уникальных названий жанров
genre_series = df['genre'].sort_values().unique()
print(genre_series)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Видим, что среди жанров встречаются неявные дубликаты с альтернативным написанием жанра: *'hip', 'hop', 'hip-hop' 'hiphop'*.

In [12]:
# устранение неявных дубликатов
df = df.replace(['hip','hop','hip-hop'], 'hiphop')

# проверка на неявные дубликаты
genre_series = df['genre'].sort_values().unique()
print(genre_series)

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Вывод**. В ходе предобработки были исправлены нарушения в стиле заголовков, преобразованы пропущенные значения (заменены на *'unknown'*), а также удалены явные и неявные дубликаты. 

## 3. Проверка гипотез

Проверим гипотезы относительно различия в прослушивании музыки у пользователей Москвы и Санкт-Петербурга: 
- Активность пользователей двух городов различается по дням недели.
- Популярность жанров в начале недели (в понедельник утром) и в конце недели (в пятницу вечером) в двух городах различается.
- В Москве более популярна поп-музыка, в то время как в Петербурге — русский рэп.

In [13]:
# подсчёт прослушиваний в каждом городе
grouped = df.groupby('city')['track'].count()
grouped

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

### 3.1. Прослушивания по дням недели

В датасете больше данных о прослушиваниях музыки пользователями из Москвы. Проверим, как различаются прослушивания по дням недели.

In [14]:
# подсчёт прослушиваний в каждый из трёх дней
grouped_day = df.groupby('day')['city'].count()
print(grouped_day)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64


В среднем пользователи из двух городов менее активны по средам. 

In [15]:
def number_tracks(day, city):
    """
    Функция для подсчета количества прослушиваний для конкретного города и дня.

    Параметры:
    day (str): День, для которого производится подсчет.
    city (str): Город, для которого производится подсчет.

    Возвращает:
    int: Количество прослушиваний для указанного города и дня.
    """
    track_list = df.loc[df.loc[:, 'day'] == day]
    track_list = track_list[track_list.loc[:, 'city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [16]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [17]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [18]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [19]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [20]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [21]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

In [22]:
# таблица с результатами
info = pd.DataFrame(data=[['Москва', 15740, 11056, 15945], ['Санкт-Петербург', 5614, 7003, 5895]], columns=['city', 'monday', 'wednesday', 'friday'])
info 

,city,monday,wednesday,friday
0,Москва,15740,11056,15945
1,Санкт-Петербург,5614,7003,5895


**Вывод**. В Москве наблюдается пик прослушиваний в понедельник и пятницу, в то время как в среду отмечается снижение активности. В Санкт-Петербурге, напротив, больше всего прослушиваний в среду.

### 3.2. Жанры музыки в начале и в конце недели

In [23]:
# получение таблиц moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow' и 'Saint-Petersburg'

moscow_general = df[df['city']=='Moscow']
spb_general = df[df['city']=='Saint-Petersburg']

In [24]:
def genre_weekday(df, day, time1, time2):
    """
    Функция для получения информации о самых популярных жанрах в указанный день и время.

    Параметры:
    df (DataFrame): Исходный датафрейм с данными.
    day (str): День недели для анализа.
    time1 (int): Начальное время интервала для анализа (включительно).
    time2 (int): Конечное время интервала для анализа (не включительно).

    Возвращает:
    Series: Топ-10 популярных жанров в указанный день и время.
    """
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time']<time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time']>time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted =  genre_df_grouped.sort_values(ascending=False) # ваш код здесь
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним Топ-10 популярных жанров для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00).

In [25]:
# вызов функции для утра понедельника в Москве
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [26]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [27]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [28]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Вывод**. Жанровые предпочтения в начале и конце недели в двух городах схожи в начале и конце недели. При этом в московский рейтинг вошёл жанр *“world”*, а в петербургский — джаз и классика. В Топ-10 Москвы вошел маркер пропусков *'unknown'*, следовательно для Москвы много пропущенных значений, которые могут исказить результаты исследования. 

### Самые популярные жанры в двух городах

In [29]:
# группировка кол-ва прослушиваний в Москве и Петербурге по жанру в порядке убывания

moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [30]:
# просмотр первых 10 строк moscow_genres
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [31]:
# просмотр первых 10 строк spb_genres
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Вывод**. Самый популярный жанр в обоих городах — поп-музыка. Рэп одинаково популярен и в Москве, и в Санкт-Петербурге. Гипотеза подтвердилась лишь частично.

## 4. Итоги исследования

В ходе исследования было установлено:

1. У пользователей в Москве и Петербурге активность различается по дням недели. В Москве пик прослушиваний приходится на понедельник и пятницу, в то время как в Санкт-Петербурге — на среду.

2. В начале и в конце недели музыкальные предпочтения в двух городах не меняются и схожи между собой. Единственное отличие: в Москве слушают музыку жанра “world”, в Петербурге — джаз и классику. При этом пропуски в столбце с жанром значительны и могли повлиять на результат исследования.

3. Самый популярный жанр в обоих городах — поп-музыка. Несмотря на то, что Петербург считается столицей русского рэпа, этот жанр одинаково популярен в обоих городах.

Таким образом, между пользователями двух городов больше сходств, чем различий.